# Test es.serch


In [3]:
from pprint import pprint
from elasticsearch import Elasticsearch
import pandas as pd

es = Elasticsearch(
    http_auth=('elastic','changeme')
)
res = es.search(index="plc-*",
                doc_type="plc",
                body={
                    "query": {
                        "range": {
                            "timestamp": {
                                "gte": "2018-01-18T06:00:00",
                                "lte": "2018-01-18T12:00:00"
                            }
                        }
                    },
                    "size":5,
                    "aggs":{
                    "Std_self": {
                        "extended_stats": {
                            "field": "Dia_C_Ydisp",
                            "missing": 0
                        }
                      }
                    },
                     "_source":["LSP","timestamp",'Dia_C_Xdisp','Dia_C_Ydisp']
                   }
                )

In [121]:
pprint(res)

{u'_shards': {u'failed': 0,
              u'skipped': 374,
              u'successful': 379,
              u'total': 395},
 u'hits': {u'hits': [{u'_id': u'-1203641805931429755',
                      u'_index': u'plc-2018.01.18',
                      u'_score': 1.0,
                      u'_source': {u'Dia_C_Xdisp': 6.8,
                                   u'Dia_C_Ydisp': 3.9,
                                   u'LSP': 160.63803100585938,
                                   u'timestamp': u'2018-01-18T06:00:49.330470'},
                      u'_type': u'document'},
                     {u'_id': u'-9018731753824431643',
                      u'_index': u'plc-2018.01.18',
                      u'_score': 1.0,
                      u'_source': {u'Dia_C_Xdisp': 6.7,
                                   u'Dia_C_Ydisp': 3.9,
                                   u'LSP': 160.80728149414062,
                                   u'timestamp': u'2018-01-18T06:00:45.930463'},
                      u'_type

In [10]:
print(res.values())
print('keys',res.keys())

[{u'hits': [{u'_score': 1.0, u'_type': u'document', u'_id': u'-1203641805931429755', u'_source': {u'LSP': 160.63803100585938}, u'_index': u'plc-2018.01.18'}, {u'_score': 1.0, u'_type': u'document', u'_id': u'-9018731753824431643', u'_source': {u'LSP': 160.80728149414062}, u'_index': u'plc-2018.01.18'}, {u'_score': 1.0, u'_type': u'document', u'_id': u'5748665870836254752', u'_source': {u'LSP': 160.63803100585938}, u'_index': u'plc-2018.01.18'}, {u'_score': 1.0, u'_type': u'document', u'_id': u'4180323035747120697', u'_source': {u'LSP': 160.7508544921875}, u'_index': u'plc-2018.01.18'}, {u'_score': 1.0, u'_type': u'document', u'_id': u'-2435515913402812972', u'_source': {u'LSP': 160.58160400390625}, u'_index': u'plc-2018.01.18'}], u'total': 19117, u'max_score': 1.0}, {u'successful': 375, u'failed': 0, u'skipped': 370, u'total': 375}, 133, {u'Std_self': {u'count': 19117, u'min': 0.0, u'sum_of_squares': 24669.811015262432, u'max': 6.199999809265137, u'sum': 6254.3001317977905, u'std_devia

In [13]:
print(len(res['hits']['hits']))

5


In [14]:
print(res['hits']['hits'][0].keys())

[u'_score', u'_type', u'_id', u'_source', u'_index']


In [15]:
print(res['hits']['hits'][0]['_source'].keys())

[u'LSP']


In [125]:
import pandas as pd
from scipy.stats import ttest_ind
def df_from_elastic_query(elastic_search_query):
    try:
        return pd.DataFrame.from_dict([item['_source'] for item in elastic_search_query['hits']['hits']])
    except:
        return pd.DataFrame()

pds=df_from_elastic_query(es.search(index="plc-*",
                doc_type="document",
                body={
                    "query": {
                        "range": {
                            "timestamp": {
                                "gte": "2018-01-18T06:00:00",
                                "lte": "2018-01-18T12:00:00"
                            }
                        }
                    },
                    "size":5,                 
                    "_source":["LSP","timestamp",'Dia_C_Xdisp','Dia_C_Ydisp']
                   }
                , size=30))
print(pds)
print('\n***************************sort values********************* \n')
print(pds.sort_values(by='timestamp'))
print('\n')
print(pds.LSP)   #same as pds['LSP']
print('\n')
print(pds[0:3])
#print(pds)
pds.to_csv('ELK.csv')
#pd.DataFrame.to_csv('file.csv', sep='\t')

    Dia_C_Xdisp  Dia_C_Ydisp         LSP                   timestamp
0           6.8          3.9  160.638031  2018-01-18T06:00:49.330470
1           6.7          3.9  160.807281  2018-01-18T06:00:45.930463
2           6.8          3.9  160.638031  2018-01-18T06:01:00.330772
3           6.8          3.9  160.750854  2018-01-18T06:00:11.829972
4           6.7          3.9  160.581604  2018-01-18T06:00:52.630279
5           6.8          3.9  160.713257  2018-01-18T06:00:31.630231
6           6.7          4.0  160.638031  2018-01-18T06:00:51.530417
7           6.8          3.9  160.882523  2018-01-18T06:00:30.529607
8           6.7          3.9  160.713257  2018-01-18T06:00:57.030574
9           6.8          3.9  160.525177  2018-01-18T06:02:03.431441
10          6.8          3.9  160.656830  2018-01-18T06:02:01.231162
11          6.8          3.9  161.540802  2018-01-18T06:01:48.031153
12          6.8          3.9  160.769669  2018-01-18T06:01:29.231053
13          6.8          3.9  160.

In [5]:
import pandas as pd
from scipy.stats import ttest_ind
es = Elasticsearch(
    http_auth=('elastic','changeme')
)

def df_from_elastic_query(elastic_search_query):
    try:
        return pd.DataFrame.from_dict([item['_source'] for item in elastic_search_query['hits']['hits']])
    except:
        return pd.DataFrame()

pds=df_from_elastic_query(es.search(index="plc-*",
                doc_type="document",
                scroll = '10m',
                body={
                    "query": {
                        "range": {
                            "timestamp": {
                                "gte": "2018-01-18T06:00:00",
                                "lte": "2018-01-18T12:00:00"
                            }
                        }
                    },
                    'size':10000,
                    "_source":["LSP","timestamp",'Dia_C_Xdisp','Dia_C_Ydisp']
                   }
                ))
#print(pds)
#print('\n***************************sort values********************* \n')
#print(pds.sort_values(by='timestamp'))
print('\n')
x=pds.LSP
print(x)   #same as pds['LSP']
#print('\n')
#print(pds[0:3])
#print(pds)
pds.to_csv('ELK.csv')
#pd.DataFrame.to_csv('file.csv', sep='\t'



0       160.638031
1       160.807281
2       160.638031
3       160.750854
4       160.581604
5       160.713257
6       160.638031
7       160.882523
8       160.713257
9       160.525177
10      160.656830
11      161.540802
12      160.769669
13      160.581604
14      160.807281
15      160.694458
16      160.581604
17      160.675644
18      160.732056
19      160.431137
20      160.619217
21      160.882523
22      160.581604
23      160.619217
24      160.769669
25      160.732056
26      160.750854
27      160.750854
28      160.750854
29      160.713257
           ...    
9970      0.902778
9971      0.846354
9972      0.921586
9973      0.902778
9974      0.996817
9975      0.883970
9976      0.921586
9977      0.959201
9978      1.053241
9979      0.996817
9980      0.865162
9981      0.846354
9982      0.789931
9983      0.921586
9984      0.902778
9985      0.940394
9986      0.996817
9987      0.789931
9988      0.865162
9989      0.752315
9990      0.883970
9991      

In [87]:
pds.loc[0]

Dia_C_Xdisp                           6.8
Dia_C_Ydisp                           3.9
LSP                               160.638
timestamp      2018-01-18T06:00:49.330470
Name: 0, dtype: object

In [94]:
pds.loc[:,['LSP','Dia_C_Xdisp']]


,LSP,Dia_C_Xdisp
0,160.638031,6.8
1,160.807281,6.7
2,160.638031,6.8
3,160.750854,6.8
4,160.581604,6.7
5,160.713257,6.8


In [102]:
pds.loc[1:2,:]

,Dia_C_Xdisp,Dia_C_Ydisp,LSP,timestamp
1,6.7,3.9,160.807281,2018-01-18T06:00:45.930463
2,6.8,3.9,160.638031,2018-01-18T06:01:00.330772


In [100]:
pds.loc[:,['LSP','Dia_C_Xdisp']]

,LSP,Dia_C_Xdisp
0,160.638031,6.8
1,160.807281,6.7
2,160.638031,6.8
3,160.750854,6.8
4,160.581604,6.7
5,160.713257,6.8


In [96]:
pds.iloc[3:5,0:3]

,Dia_C_Xdisp,Dia_C_Ydisp,LSP
3,6.8,3.9,160.750854
4,6.7,3.9,160.581604


# T-test in Pandas


In [105]:
from scipy.stats import ttest_ind
ttest_ind(pds.Dia_C_Xdisp, pds.Dia_C_Ydisp)


Ttest_indResult(statistic=135.9779393872408, pvalue=1.1358991620880811e-17)